In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0
!pip install pyspark

In [3]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Copy one wikidumps files 
import os
from pathlib import Path

project_id = 'assignment3-370517'

!gcloud config set project {project_id}

bucket_name = 'amit-chen-bucket-1'

postings_gcp_text = 'postings_gcp_text'
postings_gcp_anchor = 'postings_gcp_anchor'
postings_gcp_title = 'postings_gcp_title'
page_rank = 'pr'
doc_len = 'dl'
doc_title = 'dt'

# uncoment data you want to upload from bucket 

!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_text}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_anchor}/ . 
!gsutil -m cp -r gs://{bucket_name}/{postings_gcp_title}/ . 
!gsutil -m cp -r gs://{bucket_name}/{page_rank}/ . 
!gsutil -m cp -r gs://{bucket_name}/{doc_len}/ . 
!gsutil -m cp -r gs://{bucket_name}/{doc_title}/ . 

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

In [6]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2JsDuuUTH9LwAeBbnQqZXkbbdod_5NLnteyuKr4NCqq1Knyok

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 2.8 MB/s eta 0:00:00


# Unzip training files
upload and run below code:

In [ ]:
!unzip all_files_for_frontend.zip

# Messing Around 🦖

In [26]:
from flask import Flask, request, jsonify, render_template
from nltk.corpus import stopwords
import gzip
import os
import pickle
import pandas as pd
from inverted_index_gcp import *
from frontend_utils import *

INDEX_FILE = "index"
POSTINGS_GCP_TEXT_INDEX_FOLDER_URL = "postings_gcp_text"
POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL = "postings_gcp_anchor"
POSTINGS_GCP_TITLE_INDEX_FOLDER_URL = "postings_gcp_title"
PAGE_RANK_CSV_URL = "pr_part-00000-8b293cd5-fd79-47e7-a641-3d067da0c2b0-c000.csv.gz"
DT_PATH = "dt/dt.pkl"


# open files (inverted indexes etc...)
inverted_index_body = InvertedIndex.read_index(POSTINGS_GCP_TEXT_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_anchor = InvertedIndex.read_index(POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL, INDEX_FILE)
inverted_index_title = InvertedIndex.read_index(POSTINGS_GCP_TITLE_INDEX_FOLDER_URL, INDEX_FILE)

with open(DT_PATH, 'rb') as f:
    DT = pickle.load(f)

# with gzip.open(PAGE_RANK_CSV_URL) as f:
#     page_rank = pd.read_csv(f, header=None, index_col=0).squeeze("columns").to_dict()
#     max_pr_value = max(page_rank.values())
#     page_rank = {doc_id: rank/max_pr_value for doc_id, rank in page_rank.items()}


def search(query):
    ''' Returns up to a 100 of your best search results for the query. This is 
        the place to put forward your best search engine, and you are free to
        implement the retrieval whoever you'd like within the bound of the 
        project requirements (efficiency, quality, etc.). That means it is up to
        you to decide on whether to use stemming, remove stopwords, use 
        PageRank, query expansion, etc.

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res.append("AMIR")
    # END SOLUTION
    return res

def search_body(query):
    ''' Returns up to a 100 search results for the query using TFIDF AND COSINE
        SIMILARITY OF THE BODY OF ARTICLES ONLY. DO NOT use stemming. DO USE the 
        staff-provided tokenizer from Assignment 3 (GCP part) to do the 
        tokenization and remove stopwords. 

        To issue a query navigate to a URL like:
         http://YOUR_SERVER_DOMAIN/search_body?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of up to 100 search results, ordered from best to worst where each 
        element is a tuple (wiki_id, title).
    '''
    res = []
    if len(query) == 0:
      return res
    # tokenizing the query
    tokens = tokenize(query)
    print(tokens)

    # loading posting list with (word, (doc_id, tf))
    posting_lists = inverted_index_body.get_posting_lists(tokens, POSTINGS_GCP_TEXT_INDEX_FOLDER_URL)
    print(posting_lists)
    tf_dict = {}
    for posting in posting_lists:
        for doc_id, tf in posting:
            if doc_id in tf_dict:
                tf_dict[doc_id] += tf
            else:
                tf_dict[doc_id] = tf

    list_of_docs = sorted([(doc_id, score) for doc_id, score in tf_dict.items()], key=lambda x: x[1], reverse=True)[:100]
    print(list_of_docs)
    res = [(doc_id, DT[doc_id]) for doc_id, score in list_of_docs if doc_id in DT]
    return res

def search_title(query):
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE TITLE of articles, ordered in descending order of the NUMBER OF 
        QUERY WORDS that appear in the title. For example, a document with a 
        title that matches two of the query words will be ranked before a 
        document with a title that matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_title?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    if len(query) == 0:
      return res
    # tokenizing the query
    tokens = tokenize(query)
    print(tokens)

    # loading posting list with (word, (doc_id, tf))
    posting_lists = inverted_index_body.get_posting_lists(tokens, POSTINGS_GCP_TITLE_INDEX_FOLDER_URL)
    print(posting_lists)
    tf_dict = {}
    for posting in posting_lists:
        for doc_id, tf in posting:
            if doc_id in tf_dict:
                tf_dict[doc_id] += tf
            else:
                tf_dict[doc_id] = tf

    list_of_docs = sorted([(doc_id, score) for doc_id, score in tf_dict.items()], key=lambda x: x[1], reverse=True)
    res = [(doc_id, DT[doc_id]) for doc_id, score in list_of_docs]
    return res

def search_anchor(query):
    ''' Returns ALL (not just top 100) search results that contain A QUERY WORD 
        IN THE ANCHOR TEXT of articles, ordered in descending order of the 
        NUMBER OF QUERY WORDS that appear in anchor text linking to the page. 
        For example, a document with a anchor text that matches two of the 
        query words will be ranked before a document with anchor text that 
        matches only one query term. 

        Test this by navigating to the a URL like:
         http://YOUR_SERVER_DOMAIN/search_anchor?query=hello+world
        where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ALL (not just top 100) search results, ordered from best to 
        worst where each element is a tuple (wiki_id, title).
    '''
    res = []
    query = request.args.get('query', '')
    if len(query) == 0:
      return res
    # BEGIN SOLUTION
    tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
    tokens = [t for t in tokens if t not in all_stopwords]

    tf_dict = {}
    for token in tokens:
        added = []
        if token not in inverted_index_anchor.df:
            continue

        # loading posting list with (word, (doc_id_from, doc_id_dest))
        posting = read_posting_list(inverted_index_anchor, token, POSTINGS_GCP_ANCHOR_INDEX_FOLDER_URL+"/")
        for _, doc_id_dest in posting:
            if doc_id_dest in tf_dict:
                if doc_id_dest not in added:
                    tf_dict[doc_id_dest] += 1
            else:
                added.append(doc_id_dest)
                tf_dict[doc_id_dest] = 1

    # Sort Documents by number unique of tokens in doc
    list_of_dict = sorted([(doc_id, score) for doc_id, score in tf_dict.items()], key=lambda x: x[1], reverse=True)
    # END SOLUTION
    return list_of_dict

def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
    wiki_ids = request.get_json()
    if len(wiki_ids) == 0:
      return jsonify(res)
    # BEGIN SOLUTION
    res = [page_rank[wiki_id] for wiki_id in wiki_ids] 
    # END SOLUTION
    return jsonify(res)

def get_pageview():
    ''' Returns the number of page views that each of the provide wiki articles
        had in August 2021.

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pageview
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pageview', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    '''
    res = []
    wiki_ids = request.get_json()
    if len(wiki_ids) == 0:
      return jsonify(res)
    # BEGIN SOLUTION

    # END SOLUTION
    return jsonify(res)


print(search_body("best marvel movies"))


['best', 'marvel', 'movies']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[(18307001, 990), (18636995, 873), (16142831, 591), (14314727, 546), (3306354, 489), (4036172, 466), (14551473, 442), (7317283, 409), (4224440, 396), (14938664, 394), (3663849, 389), (9425157, 385), (7706340, 382), (303981, 341), (3362779, 333), (2794268, 319), (403585, 311), (16831542, 311), (1241849, 305), (11304669, 304), (625413, 287), (4483550, 286), (13215306, 283), (7081866, 279), (3483096, 278), (4151001, 269), (10589717, 263), (12284344, 255), (11605778, 251), (9149389, 244), (13020084, 239), (7893660, 236), (3382456, 235), (2830666, 233), (3627682, 233), (901742, 231), (3017764, 230), (1964969, 225), (833838, 223), (7369530, 220), (14030366, 218), (1074657, 214), (2845957, 213), (12523627, 212), (12839459, 212), (1488937, 211), (27306717, 211), (6236356, 209), (2453648, 207), (1856911, 205), (20966, 202), (1047719, 202), (6027734, 199), (2519765, 199), (1070235, 196), (25988203, 194), (11088709, 191), (5027882, 189), (248228, 188), (3174694, 188), (151343, 187), (2801735, 184

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))